In [ ]:
# %%capture
# import os, re
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth
# else:
#     # Do this only in Colab notebooks! Otherwise use pip install unsloth
#     import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
#     xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
#     !pip install --no-deps bitsandbytes accelerate {xformers} peft triton cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf "datasets==3.5.0" "huggingface_hub>=0.34.0" hf_transfer
#     !pip install --no-deps unsloth==2025.9.9
# !pip install transformers==4.55.4
# !pip install --no-deps trl==0.22.2 #major upgrade in trl (0.23.0) broke the trl patching

In [ ]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel, FastModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import torch
from torch import tensor
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, ModernBertModel, AutoModelForSequenceClassification, training_args
from datasets import load_dataset, Dataset
from tqdm import tqdm

model_name = 'answerdotai/ModernBERT-base'

NUM_CLASSES = 3
DATA_DIR = "data/"

%env UNSLOTH_DISABLE_FAST_GENERATION = 1

model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
    full_finetuning=True,
)
print("model parameters:" + str(sum(p.numel() for p in model.parameters())))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/torchao/quantization/quant_api.py:2525: SyntaxWarning: invalid escape sequence '\.'
  * regex for parameter names, must start with `re:`, e.g. `re:language\.layers\..+\.q_proj.weight`.


INFO 12-20 15:59:46 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-20 15:59:53 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
env: UNSLOTH_DISABLE_FAST_GENERATION=1
==((====))==  Unsloth 2025.12.8: Fast Modernbert patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model parameters:149607171


# Experiment time!!

In [ ]:
############# EXPERIMENT SETTINGS #############

FPB_OOD = False

###############################################

In [ ]:
from datasets import load_dataset

ds = load_dataset("neoyipeng/financial_reasoning_aggregated")

label_dict = {'NEUTRAL/MIXED':1,
              'NEUTRAL/MIXED/OBJECTIVE':1,
              'NEGATIVE':0,
              'POSITIVE':2}

ds = ds.filter(lambda x: x["task"] == "sentiment")

if FPB_OOD: ds = ds.filter(lambda x: x["source"] != 5) # skip fpb
# ds = ds.filter(lambda x: x["source"] != 2) # source 2 (FinGPT) sucks
# ds = ds.filter(lambda x: x["source"] != 9) # source 9 (stock sentiment) sucks

ds = ds.map(lambda ex: {
    "text":   ex["text"],
    "labels":  np.eye(NUM_CLASSES)[label_dict[ex["label"]]],
},remove_columns=["label","prompt","aspect","summary_detail","title","topic", "task","__index_level_0__"])


# Inspect the first example
print(ds["train"][0])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/21.7M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24270 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/1349 [00:00<?, ? examples/s]

Map:   0%|          | 0/13004 [00:00<?, ? examples/s]

Map:   0%|          | 0/722 [00:00<?, ? examples/s]

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

{'text': 'Colliers to announce third quarter results on November 1, 2022 --  Colliers International Group Inc. (TSX: CIGI; NASDAQ: CIGI) (“Colliers” or the “Company”) today announced that results for the third quarter ended September 30, 2022 will be issued by press release on November 1, 2022 at approximately 7:00am ET.', 'source': 4, 'labels': [0.0, 1.0, 0.0]}


In [ ]:
len(ds['train'])

13004

In [ ]:
from datasets import Dataset, concatenate_datasets

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'labels'],
        num_rows: 13004
    })
    validation: Dataset({
        features: ['text', 'source', 'labels'],
        num_rows: 722
    })
    test: Dataset({
        features: ['text', 'source', 'labels'],
        num_rows: 723
    })
})

In [ ]:
sentiment_ds_trn, sentiment_ds_val = concatenate_datasets([ds["train"]]), concatenate_datasets([ds["validation"]])

**databoost**

In [ ]:
# databoost = pd.read_csv("https://raw.githubusercontent.com/neoyipeng2018/ModernFinBERT/main/data/raw/ModernFinBERT_DataBoost_v0%20-%203.csv")
# assert len(databoost)==217

In [ ]:
# databoost_ds = Dataset.from_dict({
#     "text": databoost.Synthetic.tolist(),
#     "label":   databoost.Label.tolist(),

# })
# databoost_ds = databoost_ds.map(lambda ex: {
#     "text":   ex["text"],
#     "labels":  np.eye(NUM_CLASSES)[label_dict[ex["label"]]],
# },remove_columns=["label"])

# print(databoost_ds)

# print(f"Percentage of DataBoost vs Training Set: {len(databoost_ds)/len(sentiment_ds_trn)}")
# len(sentiment_ds_trn)

**NOISE REMOVE(?): add in similar datapoints in source 2 and 9**

In [ ]:
# denoised_ds = load_dataset("neoyipeng/financial_sentiment_aligned")

# label_dict = {'NEUTRAL/MIXED':1,
#               'NEUTRAL/MIXED/OBJECTIVE':1,
#               'NEGATIVE':0,
#               'POSITIVE':2}
# denoised_ds = denoised_ds.map(lambda ex: {
#     "text":   ex["text"],
#     "labels":  np.eye(NUM_CLASSES)[label_dict[ex["label"]]],
# },remove_columns=["label","prompt","aspect","summary_detail","title","topic", "source", "task","__index_level_0__"])


# # Inspect the first example
# print(denoised_ds["train"][0])

In [ ]:
# sentiment_ds_trn = concatenate_datasets([databoost_ds,sentiment_ds_trn, denoised_ds['train']]).shuffle(seed=42) #mixing again is important because the databoost is very targetted
# len(sentiment_ds_trn)

In [ ]:
def tokenize_function(examples): return tokenizer(examples['text'])

In [ ]:
sentiment_ds_trn = sentiment_ds_trn.map(tokenize_function, batched=True)
sentiment_ds_val = sentiment_ds_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/13004 [00:00<?, ? examples/s]

Map:   0%|          | 0/722 [00:00<?, ? examples/s]

In [ ]:
len(sentiment_ds_trn), len(sentiment_ds_val)

(13004, 722)

In [ ]:
import os
os.environ['WANDB_API_KEY'] = 'c6003ee33aa4304f3b55296330fbf7595523dcfc'

import wandb
wandb.login()

wandb.init(
    project="modernfinbert-final",
    name="fullds",
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: neoyipeng to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
N_EPOCHS = 3
LR = 5e-5

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sentiment_ds_trn,
    eval_dataset=sentiment_ds_val,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2, #default was 1 but lets make it more stable with 4
        warmup_steps=10,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim=training_args.OptimizerNames.ADAMW_TORCH,
        learning_rate=LR,
        weight_decay=0.001,
        lr_scheduler_type="cosine",
        seed=3407,

        num_train_epochs=N_EPOCHS, #started at 10 but eval accuracy seemed to peak halfway, so going to 5.

        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",  # or "eval_loss"
        greater_is_better=False,

        save_strategy="epoch",
        report_to="wandb",
        group_by_length=False,
        eval_strategy="epoch",
        # eval_steps=0.1,
        logging_strategy="epoch",
        # logging_steps=0.1,
    ),
    compute_metrics=lambda eval_pred: { "accuracy": accuracy_score(eval_pred[1].argmax(axis=-1), eval_pred[0].argmax(axis=-1)) }
)

/tmp/ipython-input-879666662.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 13,004 | Num Epochs = 3 | Total steps = 1,221
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 2 x 1) = 32
 "-____-"     Trainable parameters = 149,607,171 of 149,607,171 (100.00% trained)
Unsloth: Not an error, but ModernBertModel does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss,Accuracy
1,0.346400,0.301641,0.801939
2,0.200500,0.255430,0.853186
3,0.094500,0.344711,0.844875


In [ ]:
val_acc = input('Enter in percentage, so if 84.6%, enter 84.6 to 1 dp: ')

Enter in percentage, so if 84.6%, enter 84.6 to 1 dp: 85.3


In [ ]:
model = model.cuda()
model = model.eval()
FastLanguageModel.for_inference(model)
print()

# Load Best Model

In [ ]:
# from transformers.trainer_utils import get_last_checkpoint
# output_dir = "trainer_output"
# last_checkpoint = get_last_checkpoint(output_dir)
# print("Last checkpoint:", last_checkpoint)

ckpt = input('Enter best checkpoint: ')

model, tokenizer = FastModel.from_pretrained(
    model_name = f'trainer_output/checkpoint-{ckpt}',load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
)

Enter best checkpoint: 814
==((====))==  Unsloth 2025.12.8: Fast Modernbert patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


# test scores

In [ ]:
# # Get the test split
# test_data = ds['test']

# # Evaluate on test set
# model = model.cuda().eval()
# FastLanguageModel.for_inference(model)

# batch_size = 32
# correct = 0
# results = []

# # Access the features directly from the dataset
# test_texts = test_data['text']
# test_labels = test_data['labels']
# test_sources = test_data['source']  # Add source

# with torch.no_grad():
#     for i in tqdm(range(0, len(test_texts), batch_size), desc="Evaluating Test Set"):
#         batch_texts = test_texts[i:i+batch_size]
#         batch_labels = test_labels[i:i+batch_size]
#         batch_sources = test_sources[i:i+batch_size]  # Add source

#         # Convert one-hot labels to class indices
#         batch_labels = np.argmax(batch_labels, axis=1)

#         # Tokenize
#         inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
#                           truncation=True, max_length=2048)
#         inputs = {k: v.cuda() for k, v in inputs.items()}

#         # Forward pass
#         outputs = model(**inputs)
#         logits = outputs.logits
#         probs = F.softmax(logits, dim=-1)
#         preds = torch.argmax(probs, dim=-1).cpu().numpy()

#         # Count correct
#         correct += np.sum(preds == batch_labels)

#         # Store results
#         for j in range(len(batch_texts)):
#             results.append({
#                 "text": batch_texts[j][:200],
#                 "true": batch_labels[j],
#                 "pred": preds[j],
#                 "probs": probs[j].detach().float().cpu().numpy(),
#                 "correct": preds[j] == batch_labels[j],
#                 "source": batch_sources[j]  # Add source
#             })

# test_accuracy = 100 * correct / len(test_texts)
# print(f"\nTest Set Accuracy: {test_accuracy:.2f}% ({correct}/{len(test_texts)})")

# # Breakdown by source
# print("\n" + "="*50)
# print("Accuracy by Source:")
# print("="*50)

# results_df = pd.DataFrame(results)
# source_stats = results_df.groupby('source').agg(
#     correct=('correct', 'sum'),
#     total=('correct', 'count')
# ).reset_index()
# source_stats['accuracy'] = 100 * source_stats['correct'] / source_stats['total']
# source_stats = source_stats.sort_values('accuracy', ascending=False)

# for _, row in source_stats.iterrows():
#     print(f"{row['source']:30s} {row['accuracy']:6.2f}% ({row['correct']:4d}/{row['total']:4d})")

In [ ]:
# Get the test split
test_data = ds['test']

# Evaluate on test set
model = model.cuda().eval()
FastLanguageModel.for_inference(model)

batch_size = 32
correct = 0
results = []

# Access the features directly from the dataset
test_texts = test_data['text']
test_labels = test_data['labels']
test_sources = test_data['source']

with torch.no_grad():
    for i in tqdm(range(0, len(test_texts), batch_size), desc="Evaluating Test Set"):
        batch_texts = test_texts[i:i+batch_size]
        batch_labels = test_labels[i:i+batch_size]
        batch_sources = test_sources[i:i+batch_size]

        # Convert one-hot labels to class indices
        batch_labels = np.argmax(batch_labels, axis=1)

        # Tokenize
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
                          truncation=True, max_length=2048)
        inputs = {k: v.cuda() for k, v in inputs.items()}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1).cpu().numpy()

        # Count correct
        correct += np.sum(preds == batch_labels)

        # Store results
        for j in range(len(batch_texts)):
            results.append({
                "text": batch_texts[j][:200],
                "true": batch_labels[j],
                "pred": preds[j],
                "probs": probs[j].detach().float().cpu().numpy(),
                "correct": preds[j] == batch_labels[j],
                "source": batch_sources[j]
            })

test_accuracy = 100 * correct / len(test_texts)
print(f"\nTest Set Accuracy: {test_accuracy:.2f}% ({correct}/{len(test_texts)})")

# Breakdown by source
print("\n" + "="*50)
print("Accuracy by Source:")
print("="*50)

results_df = pd.DataFrame(results)
source_stats = results_df.groupby('source').agg(
    correct=('correct', 'sum'),
    total=('correct', 'count')
).reset_index()
source_stats['accuracy'] = 100 * source_stats['correct'] / source_stats['total']
source_stats = source_stats.sort_values('accuracy', ascending=False)

Evaluating Test Set: 100%|██████████| 23/23 [00:09<00:00,  2.31it/s]


Test Set Accuracy: 83.13% (601/723)

Accuracy by Source:


ValueError: Unknown format code 's' for object of type 'float'

In [ ]:
for _, row in source_stats.iterrows():
    print(f"{str(row['source']):30s} {row['accuracy']:6.2f}% ({int(row['correct']):4d}/{int(row['total']):4d})")

# Create formatted table for model card
source_table_rows = []
for _, row in source_stats.iterrows():
    source_table_rows.append(f"| {row['source']} | {row['accuracy']:.1f}% | {int(row['correct'])}/{int(row['total'])} |")
source_accuracy_table = "\n".join(source_table_rows)

4.0                             89.53% (  77/  86)
9.0                             87.98% ( 205/ 233)
5.0                             84.36% ( 205/ 243)
3.0                             80.00% (  20/  25)
8.0                             69.12% (  94/ 136)


In [ ]:
# # Get the test split
# test_data = ds['test']

# # Evaluate on test set
# model = model.cuda().eval()
# FastLanguageModel.for_inference(model)

# batch_size = 32
# correct = 0
# results = []

# # Access the features directly from the dataset
# test_texts = test_data['text']
# test_labels = test_data['labels']

# with torch.no_grad():
#     for i in tqdm(range(0, len(test_texts), batch_size), desc="Evaluating Test Set"):
#         batch_texts = test_texts[i:i+batch_size]
#         batch_labels = test_labels[i:i+batch_size]

#         # Convert one-hot labels to class indices
#         batch_labels = np.argmax(batch_labels, axis=1)

#         # Tokenize
#         inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
#                           truncation=True, max_length=2048)
#         inputs = {k: v.cuda() for k, v in inputs.items()}

#         # Forward pass
#         outputs = model(**inputs)
#         logits = outputs.logits
#         probs = F.softmax(logits, dim=-1)
#         preds = torch.argmax(probs, dim=-1).cpu().numpy()

#         # Count correct
#         correct += np.sum(preds == batch_labels)

#         # Store results
#         for j in range(len(batch_texts)):
#             results.append({
#                 "text": batch_texts[j][:200],
#                 "true": batch_labels[j],
#                 "pred": preds[j],
#                 "probs": probs[j].detach().float().cpu().numpy(),
#                 "correct": preds[j] == batch_labels[j]
#             })

# test_accuracy = 100 * correct / len(test_texts)
# print(f"\nTest Set Accuracy: {test_accuracy:.2f}% ({correct}/{len(test_texts)})")

Evaluating Test Set: 100%|██████████| 23/23 [00:07<00:00,  3.01it/s]


Test Set Accuracy: 82.71% (598/723)


# run on fpb

In [ ]:
# # Download the zip file from GitHub
# !wget https://github.com/neoyipeng2018/ModernFinBERT/raw/main/data/FinancialPhraseBank-v1.0.zip

# # Unzip the file
# !unzip FinancialPhraseBank-v1.0.zip

# # Now let's load and process the data
# import pandas as pd
# from datasets import Dataset, DatasetDict

# # The FinancialPhraseBank typically comes with sentences in a text file
# # Let's check what files we have
# !ls -la

# # Assuming the main file is "Sentences_AllAgree.txt" (adjust based on actual filename)
# # This file usually has format: sentence@@@sentiment
# data = []
# with open('/content/FinancialPhraseBank-v1.0/Sentences_50Agree.txt', 'r', encoding='utf-8', errors='ignore') as f:
#     for line in f:
#         if '@' in line:
#             parts = line.strip().split('@')
#             if len(parts) >= 2:
#                 sentence = parts[0].strip()
#                 sentiment = parts[-1].strip()
#                 data.append({'text': sentence, 'label': sentiment})

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Map sentiment labels to integers
# label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
# df['label'] = df['label'].map(label_map)

# # Convert to Hugging Face Dataset
# fpb = Dataset.from_pandas(df.reset_index(drop=True))

--2025-12-20 12:59:33--  https://github.com/neoyipeng2018/ModernFinBERT/raw/main/data/FinancialPhraseBank-v1.0.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/neoyipeng2018/ModernFinBERT/main/data/FinancialPhraseBank-v1.0.zip [following]
--2025-12-20 12:59:34--  https://raw.githubusercontent.com/neoyipeng2018/ModernFinBERT/main/data/FinancialPhraseBank-v1.0.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681890 (666K) [application/zip]
Saving to: ‘FinancialPhraseBank-v1.0.zip’

FinancialPhraseBank 100%[===================>] 665.91K  --.-KB/s    in 0.03s   

2025-12-20 12:59:34 (23.5 MB/s

In [ ]:
# # Evaluate on FinancialPhraseBank
# model = model.cuda().eval()
# FastLanguageModel.for_inference(model)

# batch_size = 32
# correct = 0
# results = []

# fpb_texts, fpb_labels = fpb['text'], fpb['label']

# with torch.no_grad():
#     for i in tqdm(range(0, len(fpb_texts), batch_size), desc="Evaluating FPB"):
#         batch_texts = fpb_texts[i:i+batch_size]
#         batch_labels = fpb_labels[i:i+batch_size]

#         # Tokenize
#         inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
#                           truncation=True, max_length=2048)
#         inputs = {k: v.cuda() for k, v in inputs.items()}

#         # Forward pass
#         outputs = model(**inputs)
#         logits = outputs.logits
#         probs = F.softmax(logits, dim=-1)
#         preds = torch.argmax(probs, dim=-1).cpu().numpy()

#         # Count correct
#         correct += np.sum(preds == batch_labels)

#         # Store results
#         for j in range(len(batch_texts)):
#             results.append({
#                 "text": batch_texts[j][:200],
#                 "true": batch_labels[j],
#                 "pred": preds[j],
#                 "probs": probs[j].detach().float().cpu().numpy(),
#                 "correct": preds[j] == batch_labels[j]
#             })

# accuracy = 100 * correct / len(fpb_texts)
# print(f"\nFinancialPhraseBank accuracy: {accuracy:.2f}% ({correct}/{len(fpb_texts)})")

Evaluating FPB: 100%|██████████| 152/152 [00:10<00:00, 14.44it/s]


FinancialPhraseBank accuracy: 95.40% (4623/4846)


In [ ]:
# # Show detailed results
# from sklearn.metrics import classification_report, confusion_matrix
# import random

# y_true = [r['true'] for r in results]
# y_pred = [r['pred'] for r in results]

# print("\nClassification Report:")
# print(classification_report(y_true, y_pred, target_names=['NEGATIVE', 'NEUTRAL', 'POSITIVE']))

# print("\nConfusion Matrix:")
# print(confusion_matrix(y_true, y_pred))

# # Show some random examples
# print("\n--- Random sample predictions ---")
# for sample in random.sample(results, min(10, len(results))):
#     status = "✅" if sample['correct'] else "❌"
#     print(f"\nText: {sample['text']}...")
#     print(f"True: {sample['true']} | Pred: {sample['pred']} {status}")
#     print(f"Confidence: {sample['probs'][sample['pred']]:.3f}")


Classification Report:
              precision    recall  f1-score   support

    NEGATIVE       0.95      0.95      0.95       604
     NEUTRAL       0.96      0.96      0.96      2879
    POSITIVE       0.94      0.94      0.94      1363

    accuracy                           0.95      4846
   macro avg       0.95      0.95      0.95      4846
weighted avg       0.95      0.95      0.95      4846


Confusion Matrix:
[[ 571   26    7]
 [  24 2776   79]
 [   4   83 1276]]


# save model in huggingface hub

In [ ]:
# Login to HuggingFace Hub
from huggingface_hub import login, HfApi
import os
login()  # Uncomment this line when ready to upload

In [ ]:
FPB_OOD

False

In [ ]:
if FPB_OOD:
  # Create model card and upload model
  model_name = "neoyipeng/ModernFinBERT-experiment-FPB-OOD"

  model_card_content = f"""---
language: en
license: apache-2.0
tags:
- finance
- sentiment-analysis
- modernbert
- financial-nlp
datasets:
- neoyipeng/financial_reasoning_aggregated
metrics:
- accuracy
widget:
- text: "The company reported strong quarterly earnings with revenue beating expectations."
  example_title: "Positive Example"
- text: "Stock prices fell sharply following disappointing guidance from management."
  example_title: "Negative Example"
- text: "The merger is expected to close in Q3 pending regulatory approval."
  example_title: "Neutral Example"
---

# ModernFinBERT: Financial Sentiment Analysis

ModernFinBERT is a financial sentiment analysis model based on the ModernBERT architecture, fine-tuned specifically for financial text classification.

## Model Details

- **Base Model**: answerdotai/ModernBERT-base
- **Task**: 3-class sentiment classification (Negative, Neutral, Positive)
- **Training Data**: Financial text from multiple sources (excluding FinancialPhraseBank)
- **Test Data**: FinancialPhraseBank
- **Parameters**: {sum(p.numel() for p in model.parameters()):,}

## Performance

### Comparison on FinancialPhraseBank Dataset

| Model | Out of Distribution Accuracy | Architecture | Parameters |
|-------|----------|--------------|------------|
| **ModernFinBERT** (ours) | **{accuracy:.2f}%** | ModernBERT-base | 149M |
| FinBERT-tone | 79.18% | BERT-base | 109M |

ModernFinBERT achieves state-of-the-art performance on financial sentiment analysis, outperforming the widely-used FinBERT-tone model.

## Usage

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("{model_name}")
model = AutoModelForSequenceClassification.from_pretrained("{model_name}")

text = "The company's quarterly results exceeded analyst expectations."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

labels = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
predicted_class = labels[predictions.argmax().item()]
confidence = predictions.max().item()

print(f"Sentiment: {{predicted_class}} ({{confidence:.2f}})")
```

## Training Details

- **Epochs**: {N_EPOCHS}
- **Batch Size**: 32
- **Learning Rate**: {LR}
- **Optimizer**: AdamW
- **Scheduler**: Cosine
- **Framework**: Unsloth + Transformers

## Citation

If you use this model, please cite:

```bibtex
@misc{{modernfinbert2025,
  title={{ModernFinBERT: A Modern Approach to Financial Sentiment Analysis}},
  author={{Neo Yi Peng}},
  year={{2025}},
  howpublished={{HuggingFace Model Hub}},
  url={{https://huggingface.co/{model_name}}}
}}
```
"""
  print(f"Accuracy comparison: ModernFinBERT ({accuracy:.2f}%) vs FinBERT-tone (79.18%)")

else:
  model_name = "neoyipeng/ModernFinBERT-base"

  model_card_content = f"""---
language: en
license: apache-2.0
tags:
- finance
- sentiment-analysis
- modernbert
- financial-nlp
datasets:
- neoyipeng/financial_reasoning_aggregated
metrics:
- accuracy
widget:
- text: "The company reported strong quarterly earnings with revenue beating expectations."
  example_title: "Positive Example"
- text: "Stock prices fell sharply following disappointing guidance from management."
  example_title: "Negative Example"
- text: "The merger is expected to close in Q3 pending regulatory approval."
  example_title: "Neutral Example"
---

# ModernFinBERT: Financial Sentiment Analysis

ModernFinBERT is a financial sentiment analysis model based on the ModernBERT architecture, fine-tuned specifically for financial text classification.

## Model Details

- **Base Model**: answerdotai/ModernBERT-base
- **Task**: 3-class sentiment classification (Negative, Neutral, Positive)
- **Training Data**: Vanilla sentiment tasks from multiple sources (including FinancialPhraseBank)
- **Parameters**: {sum(p.numel() for p in model.parameters()):,}

## Performance

### Overall Accuracy

| Split | Accuracy |
|-------|----------|
| Validation | {val_acc}% |
| Test | {test_accuracy:.1f}% |

### Test Accuracy by Source

| Source | Accuracy | Correct/Total |
|--------|----------|---------------|
{source_accuracy_table}

## Usage
```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("{model_name}")
model = AutoModelForSequenceClassification.from_pretrained("{model_name}")

text = "The company's quarterly results exceeded analyst expectations."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

labels = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
predicted_class = labels[predictions.argmax().item()]
confidence = predictions.max().item()

print(f"Sentiment: {{predicted_class}} ({{confidence:.2f}})")
```

## Training Details

- **Epochs**: {N_EPOCHS}
- **Batch Size**: 32
- **Learning Rate**: {LR}
- **Optimizer**: AdamW
- **Scheduler**: Cosine
- **Framework**: Unsloth + Transformers

## Citation

If you use this model, please cite:
```bibtex
@misc{{modernfinbert2025,
  title={{ModernFinBERT: A Modern Approach to Financial Sentiment Analysis}},
  author={{Neo Yi Peng}},
  year={{2025}},
  howpublished={{HuggingFace Model Hub}},
  url={{https://huggingface.co/{model_name}}}
}}
```
"""

In [ ]:
# Save model card
with open("README.md", "w") as f:
    f.write(model_card_content)

print("Model card created successfully!")


Model card created successfully!


In [ ]:
# Upload model to HuggingFace Hub
# Uncomment the lines below when ready to upload

# Push model and tokenizer to hub
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

# Upload model card
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=model_name,
    repo_type="model",
)

print(f"Model uploaded successfully to: https://huggingface.co/{model_name}")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...bis644k/model.safetensors:   0%|          |  524kB /  299MB            

Saved model to https://huggingface.co/neoyipeng/ModernFinBERT-base


README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Model uploaded successfully to: https://huggingface.co/neoyipeng/ModernFinBERT-base
